# Munging Defined 

- Shape dataset so it feeds easily into visualisation tools and machine learning models
- Involved similar set operations for most datasets 
- Cleaning: Strings changed to numbers, numbers/strings changed to dates, missing values dealt with, pruning columns 
- Transformation: Data grouped into meaningful subsets, summary statistics calculated, further filtering 

we will take you through all the usual munging steps. 

resale prices for HDBs from 2012 onwards 

Variables include: 

- Quarter and Year (need to split this variable and change format) 
- Town 
- Flat Type
- Price 

In [1]:
import pandas as pd 
import numpy as np

# First look at the data 

In [2]:
# load data 
resale = pd.read_csv("resale.csv")

In [3]:
# take a first look 
resale.head()

,quarter,town,flat_type,price
0,2007-Q2,Ang Mo Kio,1-room,na
1,2007-Q2,Ang Mo Kio,2-room,-
2,2007-Q2,Ang Mo Kio,3-room,172000
3,2007-Q2,Ang Mo Kio,4-room,260000
4,2007-Q2,Ang Mo Kio,5-room,372000


In [4]:
# how many rows and columns?  
resale.shape

(6084, 4)

In [5]:
# look for unique values 
resale["town"].unique()

array(['Ang Mo Kio', 'Bedok', 'Bishan', 'Bukit Batok', 'Bukit Merah',
       'Bukit Panjang', 'Bukit Timah', 'Central', 'Choa Chu Kang',
       'Clementi', 'Geylang', 'Hougang', 'Jurong East', 'Jurong West',
       'Kallang/Whampoa', 'Marine Parade', 'Pasir Ris', 'Punggol',
       'Queenstown', 'Sembawang', 'Sengkang', 'Serangoon', 'Tampines',
       'Toa Payoh', 'Woodlands', 'Yishun'], dtype=object)

In [6]:
resale["quarter"].unique()

array(['2007-Q2', '2007-Q3', '2007-Q4', '2008-Q1', '2008-Q2', '2008-Q3',
       '2008-Q4', '2009-Q1', '2009-Q2', '2009-Q3', '2009-Q4', '2010-Q1',
       '2010-Q2', '2010-Q3', '2010-Q4', '2011-Q1', '2011-Q2', '2011-Q3',
       '2011-Q4', '2012-Q1', '2012-Q2', '2012-Q3', '2012-Q4', '2013-Q1',
       '2013-Q2', '2013-Q3', '2013-Q4', '2014-Q1', '2014-Q2', '2014-Q3',
       '2014-Q4', '2015-Q1', '2015-Q2', '2015-Q3', '2015-Q4', '2016-Q1',
       '2016-Q2', '2016-Q3', '2016-Q4'], dtype=object)

In [7]:
# look at data type

resale[["price"]] = resale[["price"]].apply(pd.to_numeric, errors = "coerce")
resale["price"].head()

0         NaN
1         NaN
2    172000.0
3    260000.0
4    372000.0
Name: price, dtype: float64

In [8]:
# quick intro into indexing - slicing
# create a small dataset with 10 rows 
head = resale.head(10)
# access a column
head['price']

0         NaN
1         NaN
2    172000.0
3    260000.0
4    372000.0
5         NaN
6         NaN
7         NaN
8    172000.0
9    224500.0
Name: price, dtype: float64

In [9]:
# first row, keep all columns
head.iloc[1,:]

quarter         2007-Q2
town         Ang Mo Kio
flat_type        2-room
price               NaN
Name: 1, dtype: object

In [10]:
# all rows, only second column
head.iloc[:,2]

0       1-room
1       2-room
2       3-room
3       4-room
4       5-room
5    Executive
6       1-room
7       2-room
8       3-room
9       4-room
Name: flat_type, dtype: object

In [11]:
head.loc[head.price > 100000.0, :]

,quarter,town,flat_type,price
2,2007-Q2,Ang Mo Kio,3-room,172000.0
3,2007-Q2,Ang Mo Kio,4-room,260000.0
4,2007-Q2,Ang Mo Kio,5-room,372000.0
8,2007-Q2,Bedok,3-room,172000.0
9,2007-Q2,Bedok,4-room,224500.0


In [12]:
head.loc[head.price > 200000.0, ['quarter','town']]

,quarter,town
3,2007-Q2,Ang Mo Kio
4,2007-Q2,Ang Mo Kio
9,2007-Q2,Bedok


# Cleaning: Dealing with missing values, outliers and time formats 

## Missing Values 
Real world data, or data "in the wild", is rarely standardised. Often, especially with surveys, fields are not filled in and hence we have to deal with missing values. Pandas provides several tools to handle missing data. 

In [13]:
# Generate a boolean mask showing missing values 
resale_small = resale["price"].head()
resale_small.isnull()

0     True
1     True
2    False
3    False
4    False
Name: price, dtype: bool

In [14]:
# similarly, create a boolean mask, but this time showing non-missing values 
resale_small.notnull()

0    False
1    False
2     True
3     True
4     True
Name: price, dtype: bool

In [15]:
# notnull becomes usef when we use the boolean array as an index 
# here, all NaN values for price are filtered out 
resale_small[resale_small.notnull()]

2    172000.0
3    260000.0
4    372000.0
Name: price, dtype: float64

In [16]:
# dropping null values 
resale_small.dropna()


2    172000.0
3    260000.0
4    372000.0
Name: price, dtype: float64

Especially when our dataset is small, and we want to preserve information as much as we can, we don't have to drop NaN values. Instead, we can replace them with zero, and retain the information in the other columns. Even better, we can fill NaN values with an average or some other value. 

In [17]:
# filling with zero
resale_small.fillna(0)

0         0.0
1         0.0
2    172000.0
3    260000.0
4    372000.0
Name: price, dtype: float64

In [18]:
# propagate previous value backward 
resale_small.fillna(method='bfill')

0    172000.0
1    172000.0
2    172000.0
3    260000.0
4    372000.0
Name: price, dtype: float64

In [19]:
# propagate previous value forward 
resale_small.fillna(method='ffill')

0         NaN
1         NaN
2    172000.0
3    260000.0
4    372000.0
Name: price, dtype: float64

## Transforming Data with a Mapping

In [20]:
district = resale.head(23)
district

,quarter,town,flat_type,price
0,2007-Q2,Ang Mo Kio,1-room,NaN
1,2007-Q2,Ang Mo Kio,2-room,NaN
2,2007-Q2,Ang Mo Kio,3-room,172000.0
3,2007-Q2,Ang Mo Kio,4-room,260000.0
4,2007-Q2,Ang Mo Kio,5-room,372000.0
5,2007-Q2,Ang Mo Kio,Executive,NaN
6,2007-Q2,Bedok,1-room,NaN
7,2007-Q2,Bedok,2-room,NaN
8,2007-Q2,Bedok,3-room,172000.0
9,2007-Q2,Bedok,4-room,224500.0


In [21]:
map = {
    'Ang Mo Kio' : 'North East',
    'Bedok' : 'East',
    'Bukit Timah' : 'Central',
    'Bukit Batok' : 'West',
    'Bishan' : 'Central'
}

In [22]:
district['area'] = district['town'].map(map) 
district

C:\Users\najib\conda\envs\rds-demo\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,quarter,town,flat_type,price,area
0,2007-Q2,Ang Mo Kio,1-room,NaN,North East
1,2007-Q2,Ang Mo Kio,2-room,NaN,North East
2,2007-Q2,Ang Mo Kio,3-room,172000.0,North East
3,2007-Q2,Ang Mo Kio,4-room,260000.0,North East
4,2007-Q2,Ang Mo Kio,5-room,372000.0,North East
5,2007-Q2,Ang Mo Kio,Executive,NaN,North East
6,2007-Q2,Bedok,1-room,NaN,East
7,2007-Q2,Bedok,2-room,NaN,East
8,2007-Q2,Bedok,3-room,172000.0,East
9,2007-Q2,Bedok,4-room,224500.0,East


## String Operations 

In [23]:
resale['Year'], resale['Quarter'] = resale['quarter'].str.split('-',1).str
resale.head()

,quarter,town,flat_type,price,Year,Quarter
0,2007-Q2,Ang Mo Kio,1-room,NaN,2007,Q2
1,2007-Q2,Ang Mo Kio,2-room,NaN,2007,Q2
2,2007-Q2,Ang Mo Kio,3-room,172000.0,2007,Q2
3,2007-Q2,Ang Mo Kio,4-room,260000.0,2007,Q2
4,2007-Q2,Ang Mo Kio,5-room,372000.0,2007,Q2


In [24]:
# save our clean data 
resale.to_csv('clean_resale.csv')

# Drop Columns

In [25]:
# drop columns
# inplace so that original data can be modified without creating a copy.
resale.drop('quarter', axis = 1, inplace=True)
resale.head()

,town,flat_type,price,Year,Quarter
0,Ang Mo Kio,1-room,NaN,2007,Q2
1,Ang Mo Kio,2-room,NaN,2007,Q2
2,Ang Mo Kio,3-room,172000.0,2007,Q2
3,Ang Mo Kio,4-room,260000.0,2007,Q2
4,Ang Mo Kio,5-room,372000.0,2007,Q2


Other string operations include contains **count, endswith, startswith, findall, lower, upper, math, strip, etc.** 

## Dealing with time format  

In [26]:
resale['Year'].dtype

dtype('O')

In [27]:
from datetime import datetime 
resale['Year'] = pd.to_datetime(resale['Year'])
resale['Year'].dtype

dtype('<M8[ns]')

In [28]:
resale['Year'] = [x.strftime('%Y') for x in resale['Year']]
resale.head()

,town,flat_type,price,Year,Quarter
0,Ang Mo Kio,1-room,NaN,2007,Q2
1,Ang Mo Kio,2-room,NaN,2007,Q2
2,Ang Mo Kio,3-room,172000.0,2007,Q2
3,Ang Mo Kio,4-room,260000.0,2007,Q2
4,Ang Mo Kio,5-room,372000.0,2007,Q2


In [29]:
resale['Year'].dtype

dtype('O')

# Supplementing our dataset with more data: Merging and Joining Datasets 

Before we purchase a property, it's useful to take note of any amneties nearby, particularly schools. Our original dataset doesn't come with information about schools, but with Pandas, adding information to our dataset is easy with the merge( ) function. 

In [30]:
# create two data frames 
# price of 4-room for 2016 
price = pd.DataFrame({'town': ['Yishun','Woodlands','Toa Payoh','Tampines','Serangoon'],
                        'price' : [369000,350000,565000,430000,465000]})
# number of secondary schools in area in 2016
schools = pd.DataFrame({'town' : ['Woodlands','Yishun','Toa Payoh','Tampines','Serangoon'],
                        'school_num' : [10,9,6,10,4]})

In [31]:
price

,price,town
0,369000,Yishun
1,350000,Woodlands
2,565000,Toa Payoh
3,430000,Tampines
4,465000,Serangoon


In [32]:
schools

,school_num,town
0,10,Woodlands
1,9,Yishun
2,6,Toa Payoh
3,10,Tampines
4,4,Serangoon


In [33]:
# combine both data frames into one 
merged = pd.merge(price, schools)
merged 

,price,town,school_num
0,369000,Yishun,9
1,350000,Woodlands,10
2,565000,Toa Payoh,6
3,430000,Tampines,10
4,465000,Serangoon,4


Note that the pd.merge( ) function sees that each DataFrame has an 'employee' column and hence joins using this column as a key. Hence we have painlessly combines the information from the two inputs. 

For the SQL folks, pd.merge( ) allows more complex joins such as many-to-many joins, left joins, inner joins etc. See the documentation for more details, or Wes McKinney's Python for Data Analysis is a great resource. 

A quick alternative to the pd.merge( ) function is the pd.concat( ) function. This function allows us to specify whether to join the DataFrame by column or rows. 

In [34]:
pd.concat([price, schools], axis=1)

,price,town,school_num,town
0,369000,Yishun,10,Woodlands
1,350000,Woodlands,9,Yishun
2,565000,Toa Payoh,6,Toa Payoh
3,430000,Tampines,10,Tampines
4,465000,Serangoon,4,Serangoon


In [35]:
# row-wise by default 

price_2 = pd.DataFrame({'town': ['Sengkang','Sembawang'],
                        'price' : [408000,361500]})
pd.concat([price, price_2])

,price,town
0,369000,Yishun
1,350000,Woodlands
2,565000,Toa Payoh
3,430000,Tampines
4,465000,Serangoon
0,408000,Sengkang
1,361500,Sembawang


# Aggregation

After loading our dataset, cleaning up our NaN values, merging our datasets and wrangling with data types, a common next step is to look at group statistics such as the mean or median to get a better feel for our data. In general terms, we can say we want to split our dataset into interesting groups, and apply a function to each group. For this, pandas groupby facility is particularly useful, and allows us to summarise our datasets flexibly and naturally. 

Indeed, one of the reasons why SQL and relational databases are so popular is that they easily join, filter and transform datasets. However, the group operations they can perform are limited. Pandas is more expressive than SQL, and thus allows more complex grouped operations. As long as a function can accept a pandas object or NumPy array, we'll be fine. 

Here, we'll look at: 

- Performing aggregation using the Split-Apply-Combine paradigm 
- Pivot Tables 
references: Python4DataAnalysis by Wes McKinney and PythonDataScienceHandbook by Jake Van der Plas

## Split-Apply-Combine 

When dealing with groups (aggregations), we follow a process. Namely, we *split* the data, be it a column or a row, accodring to a certain *key*. Then, we *apply* a function such as a sum calculation to each group. Finally, we *combine* the results of the function into a new object. 

To make things concrete, let's look at an example, using the HDB resale dataset from before
![alt text](splitapplycombine.png)

In [36]:
resale.head()

,town,flat_type,price,Year,Quarter
0,Ang Mo Kio,1-room,NaN,2007,Q2
1,Ang Mo Kio,2-room,NaN,2007,Q2
2,Ang Mo Kio,3-room,172000.0,2007,Q2
3,Ang Mo Kio,4-room,260000.0,2007,Q2
4,Ang Mo Kio,5-room,372000.0,2007,Q2


In [37]:
# create groupby object that is ready for apply operation that follows.
grouped = resale['price'].groupby(resale['Year'])
grouped

In [38]:
# apply mean operation 
grouped.mean()

Year
2007    299387.659574
2008    345821.355932
2009    369963.897764
2010    419546.325879
2011    454066.129032
2012    485058.303887
2013    491575.918367
2014    456080.851064
2015    452604.705882
2016    460511.235955
Name: price, dtype: float64

In [39]:
# another example 
grouped = resale['price'].groupby(resale['town'])
grouped.mean()

town
Ang Mo Kio         421810.000000
Bedok              406429.921260
Bishan             513168.604651
Bukit Batok        398204.878049
Bukit Merah        530652.991453
Bukit Panjang      387819.685039
Bukit Timah                  NaN
Central            461485.714286
Choa Chu Kang      409066.666667
Clementi           400386.904762
Geylang            374831.111111
Hougang            419569.078947
Jurong East        383211.304348
Jurong West        393205.128205
Kallang/Whampoa    445631.603774
Marine Parade      367378.260870
Pasir Ris          465555.932203
Punggol            438340.740741
Queenstown         482574.157303
Sembawang          413936.538462
Sengkang           457212.280702
Serangoon          387787.500000
Tampines           447267.741935
Toa Payoh          453585.454545
Woodlands          381926.451613
Yishun             359425.190840
Name: price, dtype: float64

In [40]:
# combine keys 
grouped = resale['price'].groupby([resale['Year'], resale['town']])
grouped.mean()

Year  town           
2007  Ang Mo Kio         291977.777778
      Bedok              295454.545455
      Bishan             372737.500000
      Bukit Batok        305275.000000
      Bukit Merah        379166.666667
      Bukit Panjang      256254.545455
      Bukit Timah                  NaN
      Central            321000.000000
      Choa Chu Kang      289860.000000
      Clementi           289385.714286
      Geylang            263362.500000
      Hougang            286958.333333
      Jurong East        271380.000000
      Jurong West        256000.000000
      Kallang/Whampoa    315166.666667
      Marine Parade      291250.000000
      Pasir Ris          329033.333333
      Punggol            301583.333333
      Queenstown         391166.666667
      Sembawang          290737.500000
      Sengkang           315866.666667
      Serangoon          282272.727273
      Tampines           311375.000000
      Toa Payoh          331090.909091
      Woodlands          261583.333333
   

In [41]:
# code above allows grouping with a series not in the data frame
# if grouping information is in the same data frame, we have a shorter way:
# non-numerical column automatically dropped. 
grouped = resale.groupby('town').mean()
grouped.head()

,price
town,
Ang Mo Kio,421810.000000
Bedok,406429.921260
Bishan,513168.604651
Bukit Batok,398204.878049
Bukit Merah,530652.991453


In [42]:
# group with a mapping 
# DataFrame of property guru ratings 
import numpy as np
rating = pd.DataFrame(np.random.randint(0,5, size=(4,4)),
                  columns=['Bedok','Changi','Queenstown','Toa Payoh'],
                  index=['Tan','Dennis','Jane','Wong'])
rating

,Bedok,Changi,Queenstown,Toa Payoh
Tan,2,2,1,3
Dennis,3,0,0,0
Jane,3,2,2,1
Wong,3,3,0,1


In [43]:
map = {
    'Bedok' : 'South East',
    'Changi' : 'South East',
    'Queenstown' : 'Central',
    'Toa Payoh' : 'Central'
}

by_column = rating.groupby(map, axis=1)
by_column.mean()

,Central,South East
Tan,2.0,2.0
Dennis,0.0,1.5
Jane,1.5,2.5
Wong,0.5,3.0


In [44]:
# group with a function 
rating.groupby(len).min()

,Bedok,Changi,Queenstown,Toa Payoh
3,2,2,1,3
4,3,2,0,1
6,3,0,0,0


In [45]:
# mixing functions with arrays, dicts or Series is fine - flexibility! 
key_list = ['one','one','two','two']
rating.groupby([len,key_list]).min()

,,Bedok,Changi,Queenstown,Toa Payoh
3,one,2,2,1,3
4,two,3,2,0,1
6,one,3,0,0,0
